# JSON Evaluators

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### JsonValidityEvaluator

In [1]:
from langchain.evaluation import JsonValidityEvaluator
from langchain_openai import ChatOpenAI

evaluator = JsonValidityEvaluator()
# Equivalently
# evaluator = load_evaluator("json_validity")
prediction = '{"name": "John", "age": 30, "city": "New York"}'

result = evaluator.evaluate_strings(prediction=prediction)
print(result)

{'score': 1}


In [2]:
prediction = '{"name": "John", "age": 30, "city": "New York",}'
result = evaluator.evaluate_strings(prediction=prediction)
print(result)

{'score': 0, 'reasoning': 'Expecting property name enclosed in double quotes: line 1 column 48 (char 47)'}


### JsonEqualityEvaluator

In [3]:
from langchain.evaluation import JsonEqualityEvaluator

evaluator = JsonEqualityEvaluator()
# Equivalently
# evaluator = load_evaluator("json_equality")
result = evaluator.evaluate_strings(prediction='{"a": 1}', reference='{"a": 1}')
print(result)

{'score': True}


In [4]:
result = evaluator.evaluate_strings(prediction='{"a": 1}', reference='{"a": 2}')
print(result)

{'score': False}


In [5]:
result = evaluator.evaluate_strings(prediction={"a": 1}, reference={"a": 2})
print(result)

{'score': False}


### JsonEditDistanceEvaluator

In [8]:
from langchain.evaluation import JsonEditDistanceEvaluator

evaluator = JsonEditDistanceEvaluator()
# Equivalently
# evaluator = load_evaluator("json_edit_distance")

result = evaluator.evaluate_strings(
    prediction='{"a": 1, "b": 2}', reference='{"a": 1, "b": 8}'
)
print(result)

{'score': 0.07692307692307693}


In [9]:
# The values are canonicalized prior to comparison
result = evaluator.evaluate_strings(
    prediction="""
    {
        "b": 3,
        "a":   1
    }""",
    reference='{"a": 1, "b": 3}',
)
print(result)

{'score': 0.0}


In [10]:
# リストは順序を維持する。しかし、
result = evaluator.evaluate_strings(
    prediction='{"a": [1, 2]}', reference='{"a": [2, 1]}'
)
print(result)

{'score': 0.18181818181818182}


In [11]:
# オブジェクトを直接渡すこともできます。
result = evaluator.evaluate_strings(prediction={"a": 1}, reference={"a": 2})
print(result)

{'score': 0.14285714285714285}


### JsonSchemaEvaluator

In [12]:
from langchain.evaluation import JsonSchemaEvaluator

evaluator = JsonSchemaEvaluator()
# Equivalently
# evaluator = load_evaluator("json_schema_validation")

result = evaluator.evaluate_strings(
    prediction='{"name": "John", "age": 30}',
    reference={
        "type": "object",
        "properties": {"name": {"type": "string"}, "age": {"type": "integer"}},
    },
)
print(result)

{'score': True}


In [13]:
result = evaluator.evaluate_strings(
    prediction='{"name": "John", "age": 30}',
    reference='{"type": "object", "properties": {"name": {"type": "string"}, "age": {"type": "integer"}}}',
)
print(result)

{'score': True}


In [14]:
result = evaluator.evaluate_strings(
    prediction='{"name": "John", "age": 30}',
    reference='{"type": "object", "properties": {"name": {"type": "string"},'
    '"age": {"type": "integer", "minimum": 66}}}',
)
print(result)

{'score': False, 'reasoning': "<ValidationError: '30 is less than the minimum of 66'>"}
